# CSI5155 Machine Learning Assignment 2

This notebook serves as a starting point to firstly evaluate the predictions from models which we trained in assignment 1, and precisely to calculate the SHAP values for each selected classifiers.

# Installing Packages and Prerequisites

In [ ]:
import numpy as np;
import constants;
import os;
from fileOrganizer import unpack;
import random;
from explainer import explainer;
import gc;

c:\Users\kelvi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model selection and Applying SHAP

According to assignment 1 (as shown in this notebook `CSI5155 Assignment 1 Evaluation Part - Kelvin Mock 300453668.ipynb` and the report `CSI5155 Assignment 1 Report.pdf`), we compared the Areas Under the Curves (AUCs) among 6 models. We also concluded in the report that the AUC is a great metric to inference the overall accuracy of a model. For simplicity, we only consider the **original** models where sampling techniques are not applied. We have the following conclusions:
- Based on the Chocolate dataset, the best classifier (with the largest AUC) is the Decision Tree classifier; and, 
- Based on the Magic Mushroom dataset, the best classifier (with the largest AUC) is the Multi-Layer Perceptron (MLP) classifier,

whereas,

- Based on the Chocolate dataset, the worst classifier (with the lowest AUC) is the Support Vector Machine (SVM) classifier; and, 
- Based on the Magic Mushroom dataset, the worst classifier (with the lowest AUC) is also the SVM classifier. 

## Loading the Models from files

In [ ]:
def load_models(path: str):
    # Choco Best: Decision Tree
    choco_bestModel = unpack(os.path.join(path, constants.filepaths["choc_posttrained_decisionTree"]));
    # Choco Worst: SVM
    choco_worstModel = unpack(os.path.join(path, constants.filepaths["choc_posttrained_SVC"]));
    # Mushroom Best: MLP
    mush_bestModel = unpack(os.path.join(path, constants.filepaths["mushrooms_posttrained_MLP"]));
    # Mushroom Worst: SVM
    mush_worstModel = unpack(os.path.join(path, constants.filepaths["mushrooms_posttrained_SVC"]));
    return choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel;

try:
    choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel = load_models(constants.ASM1_DIR);
except FileNotFoundError:
    # alt path for remote / github action
    choco_bestModel, choco_worstModel, mush_bestModel, mush_worstModel = load_models(constants.ASM1_DIR_ALT);

### Inspecting the Imported Models

In [ ]:
choco_bestModel

RandomizedSearchCV(estimator=DecisionTreeClassifier(criterion='entropy',
                                                    splitter='random'),
                   param_distributions={'max_depth': array([10, 19, 12, 14, 19, 18, 10,  4,  6, 19, 15,  1, 14,  3,  5, 14, 17,
       16,  2,  7, 19, 11, 16, 17,  4,  8, 13, 13,  7, 16, 17,  5, 14, 13,
        9,  5,  8, 14, 11, 14, 18,  1,  6,  7, 16, 11, 18, 16, 17, 18, 15,
       16,  9, 12,  1, 11,  8,  3, 15, 15, 17, 18,  3,  3,  3, 10,  5,  2,
        3,  5, 17,  6, 18, 18,  9, 19,  8,  4,  1, 14, 18,  7, 13, 17, 14,
        4, 11,  2,  2,  4, 15, 14, 13,  2,  3,  5,  3,  3, 13, 17],
      dtype=int64),
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021317376680>},
                   random_state=42)

Since the Decision Tree classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [ ]:
choco_bestModel = choco_bestModel.best_estimator_;
choco_bestModel

DecisionTreeClassifier(criterion='entropy', max_depth=16,
                       min_samples_split=0.9556428757689246, splitter='random')

In [ ]:
choco_worstModel

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002135F3653C0>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000213173772E0>,
                                        'gamma': ['scale', 'auto',
                                                  0.06312872320515445,
                                                  0.08534783009051013,
                                                  0.07101331585257817,
                                                  0.07840208005724572,
                                                  0.04723202725171398],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42)

Since the SVM classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [ ]:
choco_worstModel = choco_worstModel.best_estimator_;
choco_worstModel

SVC(C=3.4370861113902182, degree=4, gamma=0.07840208005724572, kernel='linear')

In [ ]:
mush_bestModel

MLPClassifier(hidden_layer_sizes=3769, learning_rate='adaptive', max_iter=500)

In [ ]:
mush_worstModel

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000213173A7F40>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000213173A7670>,
                                        'gamma': ['scale', 'auto',
                                                  0.06312872320515445,
                                                  0.08534783009051013,
                                                  0.07101331585257817,
                                                  0.07840208005724572,
                                                  0.04723202725171398],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42)

Since the SVM classifier has been optimized by RandomizedSearchCV, we need to extract the estimator. 

In [ ]:
mush_worstModel = mush_worstModel.best_estimator_;
mush_worstModel

SVC(C=3.4370861113902182, degree=4, gamma=0.07840208005724572, kernel='linear')

## Loading the Datasets from files

In [ ]:
def load_datasets(path: str):
    # Choco train set
    choco_X_train = unpack(os.path.join(path, constants.filepaths["choc_train-set_samples"]));
    choco_y_train = unpack(os.path.join(path, constants.filepaths["choc_train-set_labels"]));

    # mushroom train set
    mush_X_train = unpack(os.path.join(path, constants.filepaths["mushrooms_train-set_samples"]));
    mush_y_train = unpack(os.path.join(path, constants.filepaths["mushrooms_train-set_labels"]));

    # Choco test set
    choco_X_test = unpack(os.path.join(path, constants.filepaths["choc_test-set_samples"]));
    choco_y_test = unpack(os.path.join(path, constants.filepaths["choc_test-set_labels"]));
    # mushroom test set
    mush_X_test = unpack(os.path.join(path, constants.filepaths["mushrooms_test-set_samples"]));
    mush_y_test = unpack(os.path.join(path, constants.filepaths["mushrooms_test-set_labels"]));

    return choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test;

try:
    choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test = load_datasets(constants.ASM1_DIR);
except FileNotFoundError:
    # Using ALT paths for remote / github action
    choco_X_train, choco_y_train, mush_X_train, mush_y_train, choco_X_test, choco_y_test, mush_X_test, mush_y_test = load_datasets(constants.ASM1_DIR_ALT);

### Inspecting the Imported Data

In [ ]:
print("-----Chocolate Dataset Training Set-----");
print(f"Size of the samples array in the training set from Chocolate dataset: {len(choco_X_train)}");
print(f"Size of the labels array in the training set from Chocolate dataset: {len(choco_y_train)}");
print(f"Number of features in a sample in the training set from Chocolate dataset: {len(choco_X_train[random.randint(0, len(choco_X_train)-1)])}");
unique, counts = np.unique(choco_y_train, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Mushroom Dataset Training Set-----");
print(f"Size of the samples array in the test set from Mushroom dataset: {len(mush_X_train)}");
print(f"Size of the labels array in the test set from Mushroom dataset: {len(mush_y_train)}");
print(f"Number of features in a sample in the test set from Mushroom dataset: {len(mush_X_train[random.randint(0, len(mush_X_train)-1)])}");
unique, counts = np.unique(mush_y_train, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Chocolate Dataset Test Set-----");
print(f"Size of the samples array in the test set from Chocolate dataset: {len(choco_X_test)}");
print(f"Size of the labels array in the test set from Chocolate dataset: {len(choco_y_test)}");
print(f"Number of features in a sample in the test set from Chocolate dataset: {len(choco_X_test[random.randint(0, len(choco_X_test)-1)])}");
unique, counts = np.unique(choco_y_test, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

print();

print("-----Mushroom Dataset Test Set-----");
print(f"Size of the samples array in the test set from Mushroom dataset: {len(mush_X_test)}");
print(f"Size of the labels array in the test set from Mushroom dataset: {len(mush_y_test)}");
print(f"Number of features in a sample in the test set from Mushroom dataset: {len(mush_X_test[random.randint(0, len(mush_X_test)-1)])}");
unique, counts = np.unique(mush_y_test, return_counts=True);
for i in range(len(unique)):
    print(f"Label '{unique[i]}' has: {counts[i]} samples.");

-----Chocolate Dataset Training Set-----
Size of the samples array in the training set from Chocolate dataset: 1256
Size of the labels array in the training set from Chocolate dataset: 1256
Number of features in a sample in the training set from Chocolate dataset: 13
Label 'non-user' has: 27 samples.
Label 'user' has: 1229 samples.

-----Mushroom Dataset Training Set-----
Size of the samples array in the test set from Mushroom dataset: 1256
Size of the labels array in the test set from Mushroom dataset: 1256
Number of features in a sample in the test set from Mushroom dataset: 13
Label 'non-user' has: 805 samples.
Label 'user' has: 451 samples.

-----Chocolate Dataset Test Set-----
Size of the samples array in the test set from Chocolate dataset: 629
Size of the labels array in the test set from Chocolate dataset: 629
Number of features in a sample in the test set from Chocolate dataset: 13
Label 'non-user' has: 8 samples.
Label 'user' has: 621 samples.

-----Mushroom Dataset Test Set-

## Apply SHAP Method

A SHAP value is used to represent the impact of each feature on the models’ predictions.

### Instantiating Explainers

In [ ]:
treeexmplainer_choco = explainer(
    model=choco_bestModel, # Decision Tree
    data=choco_X_train,
    modelType="tree"
)
treeexmplainer_choco

Start instantiating an explainer.
A tree explainer is instantiated successfully.


In [ ]:
linearExplainer_choco = explainer(
    model=choco_worstModel, # SVM
    data=choco_X_train,
    modelType="svm"
);
linearExplainer_choco

Start instantiating an explainer.
A Linear explainer is instantiated successfully.


In [ ]:
permExplainer_mush_best = explainer(
    model=mush_bestModel, # MLP
    data=mush_X_train,
    modelType="neural"
)
permExplainer_mush_best

Start instantiating an explainer.
A Permutation Kernel explainer is instantiated successfully.


In [ ]:
linearExplainer_mush_worst = explainer(
    model=mush_worstModel, # SVM
    data=mush_X_train,
    modelType="svm"
);
linearExplainer_mush_worst

Start instantiating an explainer.
A Linear explainer is instantiated successfully.


### Calculating SHAP values

In [ ]:
# Decision Tree
SHAP_choco_best = treeexmplainer_choco.explain(
    X_test=choco_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_choco_best.shape}");
print(f"The SHAP value for a sample: \n{SHAP_choco_best[random.randint(0, len(SHAP_choco_best)-1)]}");

A tree explainer is found.
Shape of the SHAP values set results: (629, 13, 2)
The SHAP value for a sample: 
[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.00432326  0.00432325]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]


In [ ]:
# SVM
SHAP_choco_worst = linearExplainer_choco.explain(
    X_test=choco_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_choco_worst.shape}");
print(f"The SHAP value for a sample: \n{SHAP_choco_worst[random.randint(0, len(SHAP_choco_worst)-1)]}");

A Linear explainer is found
Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[ 1.59870322e-04 -8.74319259e-06 -4.49127260e-06  2.22519157e-05
 -6.56915009e-05  7.30367327e-07 -7.21286956e-05 -6.13808176e-05
 -8.82862851e-05  1.04152867e-04 -2.94165267e-05 -1.64337848e-05
  9.99649849e-09]


In [ ]:
# MLP
SHAP_mush_best = permExplainer_mush_best.explain(
    X_test=mush_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_mush_best.shape}");
print(f"The SHAP value for a sample: \n{SHAP_mush_best[random.randint(0, len(SHAP_mush_best)-1)]}");

A Permutation Explainer is found


PermutationExplainer explainer: 630it [02:22,  4.21it/s]                         

Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# SVM
SHAP_mush_worst = linearExplainer_mush_worst.explain(
    X_test=mush_X_test
);
print(f"Shape of the SHAP values set results: {SHAP_mush_worst.shape}");
print(f"The SHAP value for a sample: \n{SHAP_mush_worst[random.randint(0, len(SHAP_mush_worst)-1)]}");

A Linear explainer is found
Shape of the SHAP values set results: (629, 13)
The SHAP value for a sample: 
[ 1.82245958e+01  3.75229786e-01 -8.17115302e-02 -3.17882357e-02
  7.04036732e-01  2.39742039e-02  2.62196096e-02 -4.84237367e-02
  6.64137134e-01  1.38074209e-02  2.54824737e-02 -8.33295762e-02
 -5.69835429e-02]


### Store the SHAP values after calculation

In [ ]:
np.save("SHAP_choco_best.npy", SHAP_choco_best);
np.save("SHAP_choco_worst.npy", SHAP_choco_worst);
np.save("SHAP_mush_best.npy", SHAP_mush_best);
np.save("SHAP_mush_worst.npy", SHAP_mush_worst);

#### Verify if the Data Dump is successful

In [ ]:
test = SHAP_choco_best == np.load("SHAP_choco_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = SHAP_choco_worst == np.load("SHAP_choco_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = SHAP_mush_best == np.load("SHAP_mush_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = SHAP_mush_worst == np.load("SHAP_mush_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


### Derive and Store the base values for plotting

In [ ]:
baseVal_choco_best = treeexmplainer_choco.calBaseVal();
baseVal_choco_best

A tree explainer is found.


0.4999999999999996

In [ ]:
baseVal_choco_worst = linearExplainer_choco.calBaseVal();
baseVal_choco_worst

A Linear explainer is found


1.0013511252644252

In [ ]:
baseVal_mush_best = permExplainer_mush_best.calBaseVal(
    model=mush_bestModel,
    X_train=mush_X_train
);
baseVal_mush_best

A Permutation Explainer is found


0.0

In [ ]:
baseVal_mush_worst = linearExplainer_mush_worst.calBaseVal();
baseVal_mush_worst

A Linear explainer is found


29.414241247512308

In [ ]:
np.save("baseVal_choco_best.npy", baseVal_choco_best);
np.save("baseVal_choco_worst", baseVal_choco_worst);
np.save("baseVal_mush_best", baseVal_mush_best);
np.save("baseVal_mush_worst.npy", baseVal_mush_worst);

#### Verify if the Data Dump is successful

In [ ]:
test = baseVal_choco_best == np.load("baseVal_choco_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = baseVal_choco_worst == np.load("baseVal_choco_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = baseVal_mush_best == np.load("baseVal_mush_best.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


In [ ]:
test = baseVal_mush_worst == np.load("baseVal_mush_worst.npy");
print(f"Number of inconsistencies from data dump: {len(test[test == False])}");

Number of inconsistencies from data dump: 0


We have made sure that all the data are dumped to .pkl files (and unpacked) accurately. Therefore, we can proceed with summary plots on another notebook called `CSI5155 Assignment 2 Plots - Kelvin Mock 300453668.ipynb`.